In [1]:
import pandas as pd
import datetime
from datetime import date
import numpy as np
import codecs

In [2]:
link = '/Users/oat/Desktop/งาน/tick/new data/2021.11.09.csv'

In [3]:
f = codecs.open(link, "r", "utf-16")

In [4]:
data = pd.read_csv(f ,sep = '\t',skip_blank_lines=True)

In [5]:
list_old_col = data.columns.to_list()
list_col = range(4)
status = 'close'
data.rename(columns={list_old_col[i]:list_col[i] for i in range(len(list_col))},inplace=True)


In [6]:
def s(x,tp,sl):
    if x < 2:
        #print(0.01)
        TP = x+(tp*0.01)
        SL = x-(sl*0.01)


    elif 2<= x < 5:
        #print(0.02)
        TP = x+(tp*0.02)
        SL = x-(sl*0.02)


    elif 5<= x < 10:
        #print(0.05)
        TP = x+(tp*0.05)
        SL = x-(sl*0.05)

    elif 10<= x < 25:
        #print(0.1)
        TP = x+(tp*0.1)
        SL = x-(sl*0.1)


    elif 25<= x < 100:
        #print(0.25)
        TP = x+(tp*0.25)
        SL = x-(sl*0.25)


    elif 100<= x < 200:
        #print(0.5)
        TP = x+(tp*0.5)
        SL = x-(sl*0.5)


    elif 200<= x < 400:
        #print(1)
        TP = x+(tp)
        SL = x-(sl)


    elif x >= 400:
        #print(2)
        TP = x+(tp*2)
        SL = x-(sl*2)

        
    #x.loc[i,'TP'] = TP
    #x.loc[i,'SL'] = SL
        
    return TP,SL

In [7]:
def Accum(df,Accum,tp,sl):
    ticket = ''
    type_sell = ''
    df['Value'] = df[1]*df[2]
    data = df.iloc[1:,:].copy()

    for j in range(len(data)):
        if data.iloc[j,3] == 'SELL':
            data.iloc[j,3] = np.nan
    
    data = data.dropna().reset_index().drop(columns='index')
    time_buy = 0

    
    for i in range(len(data)):
        
        a = datetime.datetime.strptime(data.iloc[i-1,0], '%Y.%m.%d %H:%M.%f ').strftime('%H:%M')
        b = datetime.datetime.strptime(data.iloc[i,0], '%Y.%m.%d %H:%M.%f ').strftime('%H:%M')
        c = datetime.datetime.strptime(data.iloc[i,0], '%Y.%m.%d %H:%M.%f ').strftime('%M')
        close = data.iloc[-1,1]
        time_close = data.iloc[-1,0]
       
        if a == b:
            data.loc[i,'Accum'] = data.loc[i-1,'Accum']+data.iloc[i,4]
        else:
            data.loc[i,'Accum'] = data.iloc[i,4]
  
        if data.loc[i,'Accum'] >= Accum and b != ticket:
            data.loc[i,f'BUY_{Accum}'] = 'BUY'
            value = data.iloc[i,1]
            ticket = b
            time_buy += 1
            TP,SL = s(data.iloc[i,1],tp,sl)[0],s(data.iloc[i,1],tp,sl)[1]
            data.loc[i,f'TP_{Accum}'] = TP
            data.loc[i,f'SL_{Accum}'] = SL
            
            Tick = 1
            j = i
            while Tick == 1 and j<len(data):
                
                if data.iloc[j,1] >= TP:
                    Timeticket = data.iloc[j,0]
                    sellat = TP
                    type_sell = 'TP'
                    Tick = 0
                elif data.iloc[j,1] <= SL:
                    Timeticket = data.iloc[j,0]
                    sellat = SL
                    type_sell = 'SL'
                    Tick = 0
                elif data.iloc[j,0] == time_close:
                    sellat = close
                    Timeticket = time_close
                    type_sell = 'Close'
                    Tick = 0
                else:
                    j += 1
                    
                    
            if type_sell == '':
                pass
            else:
                data.loc[i,f'SELL_Time_{Accum}'] = Timeticket
                data.loc[i,f'SELL_Price_{Accum}'] =sellat
                data.loc[i,f'Type_sell_{Accum}'] = type_sell
                data.loc[i,f'Result_{Accum}'] = sellat - data.iloc[i,1]
            
    W = data.copy()
    for i in range(len(W)):
        if data.loc[i,f'Result_{Accum}'] > 0:
                W.loc[i,'WIN_value'] = data.loc[i,f'Result_{Accum}']
        elif data.loc[i,f'Result_{Accum}'] < 0:
            W.loc[i,'LOSS_value'] = data.loc[i,f'Result_{Accum}']
            
    dfo = data.copy()
    dfo.dropna(inplace = True)
    dfo = dfo.reset_index().drop(columns = 'index')
    
    count = 0
    for i in range(1,len(dfo)):
        if dfo.loc[i,f'SELL_Time_{Accum}'] == dfo.loc[i-1,f'SELL_Time_{Accum}']:
            count += 1
            dfo.loc[i,'count'] = count
        else:
            count = 0
            pass
    PT = dfo['count'].max()
    win = W['WIN_value'].sum()
    loss = W['LOSS_value'].sum()
    result = pd.DataFrame(index = range(1),columns = ['Date','ไม้มากสุด','win_value','loss_value','P/L'])
    result.iloc[0,1] = PT
    result.iloc[0,2] = win
    result.iloc[0,3] = abs(loss)
    result.iloc[0,4] = win/abs(loss)
    return result

In [ ]:
def T0_excel(price,TP,SL):
    Finish_it = pd.ExcelWriter(f'/Users/oat/Desktop/งาน/tick/result/Conclude_{price}_{SL}.xlsx')
    x = Accum(df1,price,TP,SL)
    y = Accum(df2,price,TP,SL)
    z = Accum(df3,price,TP,SL)
    a = Accum(df4,price,TP,SL)
    b = Accum(df5,price,TP,SL)
    c = Accum(df6,price,TP,SL)
    x.to_excel(Finish_it,sheet_name='16.11')
    y.to_excel(Finish_it,sheet_name='9.11')
    z.to_excel(Finish_it,sheet_name='10.11')
    a.to_excel(Finish_it,sheet_name='11.11')
    b.to_excel(Finish_it,sheet_name='12.11')
    c.to_excel(Finish_it,sheet_name='15.11')
    Finish_it.close()
    return 'GG'

In [ ]:
Finish_it = pd.ExcelWriter('/Users/oat/Desktop/งาน/tick/result/Conclude_10M_5.xlsx')

In [ ]:
x = Accum(df1,1000000,3,4)

In [ ]:
y = Accum(df2,1000000,3,4)

In [ ]:
z = Accum(df3,1000000,3,4)

In [ ]:
a = Accum(df4,1000000,3,4)

In [ ]:
b = Accum(df5,1000000,3,4)

In [ ]:
c = Accum(df6,1000000,3,4)

In [ ]:
df1 = data['Sheet1'] #16
df2 = data['9.11']
df3 = data['10.11']
df4 = data['11.11']
df5 = data['12.11']
df6 = data['15.11']

In [ ]:
x.to_excel(Finish_it,sheet_name='16.11')
y.to_excel(Finish_it,sheet_name='9.11')
z.to_excel(Finish_it,sheet_name='10.11')
a.to_excel(Finish_it,sheet_name='11.11')
b.to_excel(Finish_it,sheet_name='12.11')
c.to_excel(Finish_it,sheet_name='15.11')

In [ ]:
Finish_it.close()

In [ ]:
a = datetime.datetime.strptime(x.iloc[7,0], '%Y.%m.%d %H:%M.%f ').strftime('%H:%M')
b = datetime.datetime.strptime(x.iloc[8,0], '%Y.%m.%d %H:%M.%f ').strftime('%M')

In [ ]:
y.to_excel('/Users/oat/Desktop/งาน/tick/result/TesT.xlsx')